In [7]:
pip install scikit-learn joblib tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
import os, json, time
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss, classification_report, confusion_matrix
from joblib import dump

# =========================
# CONFIG (EDIT THESE)
# =========================
CSV_PATH = r"C:\Users\Timothy.Mandingwa\Desktop\live_app_rules\data\EURUSD_OANDA_M1.csv"
OUT_DIR = "artifacts"

BUY_TH  = 0.60
SELL_TH = 0.40

os.makedirs(OUT_DIR, exist_ok=True)

def log(msg):
    print(f"[{time.strftime('%H:%M:%S')}] {msg}")

def ensure_dt_index(df, time_col="time"):
    if time_col in df.columns:
        df[time_col] = pd.to_datetime(df[time_col], utc=True, errors="coerce")
        df = df.dropna(subset=[time_col]).set_index(time_col)
    else:
        df.index = pd.to_datetime(df.index, utc=True, errors="coerce")
        df = df[~df.index.isna()]
    return df.sort_index()

def mid_ohlc_from_oanda(df):
    needed = ["bid_o","bid_h","bid_l","bid_c","ask_o","ask_h","ask_l","ask_c"]
    missing = [c for c in needed if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns in CSV: {missing}")

    out = pd.DataFrame(index=df.index)
    out["open"]  = (df["bid_o"].astype(float) + df["ask_o"].astype(float)) / 2.0
    out["high"]  = (df["bid_h"].astype(float) + df["ask_h"].astype(float)) / 2.0
    out["low"]   = (df["bid_l"].astype(float) + df["ask_l"].astype(float)) / 2.0
    out["close"] = (df["bid_c"].astype(float) + df["ask_c"].astype(float)) / 2.0
    out["spread"] = (df["ask_c"].astype(float) - df["bid_c"].astype(float))
    out["volume"] = pd.to_numeric(df["volume"], errors="coerce").fillna(0.0) if "volume" in df.columns else 0.0
    return out

def resample_tf(m1_mid, tf):
    tf = tf.upper().strip()
    rule = {"M15":"15min", "M30":"30min"}[tf]
    o = m1_mid["open"].resample(rule).first()
    h = m1_mid["high"].resample(rule).max()
    l = m1_mid["low"].resample(rule).min()
    c = m1_mid["close"].resample(rule).last()
    v = m1_mid["volume"].resample(rule).sum()
    sp = m1_mid["spread"].resample(rule).mean()
    return pd.DataFrame({"open":o,"high":h,"low":l,"close":c,"volume":v,"spread":sp}).dropna()

def ema(s, n): 
    return s.ewm(span=n, adjust=False).mean()

def rsi(close, n=14):
    d = close.diff()
    up = d.clip(lower=0).rolling(n).mean()
    dn = (-d.clip(upper=0)).rolling(n).mean()
    rs = up / (dn + 1e-12)
    return 100 - (100 / (1 + rs))

def atr(df, n=14):
    high, low, close = df["high"], df["low"], df["close"]
    prev = close.shift(1)
    tr = pd.concat([(high-low), (high-prev).abs(), (low-prev).abs()], axis=1).max(axis=1)
    return tr.rolling(n).mean()

def macd(close, fast=12, slow=26, sig=9):
    macd_line = ema(close, fast) - ema(close, slow)
    sig_line = ema(macd_line, sig)
    hist = macd_line - sig_line
    return macd_line, sig_line, hist

def build_features(c):
    df = c.copy()
    df["ret1"] = df["close"].pct_change(1)
    df["ret4"] = df["close"].pct_change(4)
    df["ret8"] = df["close"].pct_change(8)

    df["ema20"] = ema(df["close"], 20)
    df["ema50"] = ema(df["close"], 50)
    df["ema_diff"] = (df["ema20"] - df["ema50"]) / (df["close"].abs() + 1e-12)

    df["rsi14"] = rsi(df["close"], 14)

    m, s, h = macd(df["close"], 12, 26, 9)
    df["macd"] = m
    df["macd_sig"] = s
    df["macd_hist"] = h

    df["macd_cross_up"] = ((df["macd"].shift(1) <= df["macd_sig"].shift(1)) & (df["macd"] > df["macd_sig"])).astype(int)
    df["macd_cross_dn"] = ((df["macd"].shift(1) >= df["macd_sig"].shift(1)) & (df["macd"] < df["macd_sig"])).astype(int)

    df["atr14"] = atr(df, 14)
    df["atr_pct"] = df["atr14"] / (df["close"].abs() + 1e-12)

    df["hl_range"] = (df["high"] - df["low"]) / (df["close"].abs() + 1e-12)
    df["oc_range"] = (df["close"] - df["open"]) / (df["close"].abs() + 1e-12)

    df["spread_pct"] = df["spread"] / (df["close"].abs() + 1e-12)
    df["vol_z"] = (df["volume"] - df["volume"].rolling(100).mean()) / (df["volume"].rolling(100).std() + 1e-12)

    return df.dropna()

FEATURE_COLS = [
    "ret1","ret4","ret8",
    "ema_diff","rsi14",
    "macd","macd_sig","macd_hist","macd_cross_up","macd_cross_dn",
    "atr_pct","hl_range","oc_range",
    "spread_pct","vol_z",
]

def make_label(df, H):
    fut = df["close"].shift(-H)
    return (fut > df["close"]).astype(int)

def time_split_idx(n, train_frac=0.70, val_frac=0.15):
    tr_end = int(n * train_frac)
    va_end = int(n * (train_frac + val_frac))
    return slice(0,tr_end), slice(tr_end,va_end), slice(va_end,n)

def gating_stats(p_up, y_true, buy_th=0.60, sell_th=0.40):
    p_up = np.asarray(p_up)
    y_true = np.asarray(y_true).astype(int)

    take_buy = p_up >= buy_th
    take_sell = p_up <= sell_th
    take = take_buy | take_sell

    if take.sum() == 0:
        return {"trades":0, "win_rate":0.0}

    pred_dir = np.where(take_buy, 1, 0)
    wins = (pred_dir[take] == y_true[take])
    return {"trades": int(take.sum()), "win_rate": float(wins.mean() * 100.0)}

def train_tf(tf, H):
    log(f"STEP 1/6: Load CSV for {tf} (H={H})")
    raw = pd.read_csv(CSV_PATH)
    raw = ensure_dt_index(raw, "time")
    mid = mid_ohlc_from_oanda(raw)

    log(f"STEP 2/6: Resample -> {tf}")
    candles = resample_tf(mid, tf)
    log(f"   {tf} candles: {len(candles):,}")

    log("STEP 3/6: Features + Labels")
    feats = build_features(candles)
    feats["y"] = make_label(feats, H)
    feats = feats.dropna(subset=["y"])
    feats["y"] = feats["y"].astype(int)

    X = feats[FEATURE_COLS].astype(float).values
    y = feats["y"].values

    tr, va, te = time_split_idx(len(feats))
    Xtr, ytr = X[tr], y[tr]
    Xva, yva = X[va], y[va]
    Xte, yte = X[te], y[te]

    log(f"   Split: train={len(ytr):,} val={len(yva):,} test={len(yte):,}")

    # Candidate runs
    runs = []
    for params in ParameterGrid({"C":[0.25,0.5,1.0,2.0,4.0]}):
        runs.append(("logreg", params))
    for params in ParameterGrid({"n_estimators":[300,600], "max_depth":[6,8,10], "min_samples_leaf":[25,50,100]}):
        runs.append(("rf", params))
    for params in ParameterGrid({"max_depth":[3,6], "learning_rate":[0.03,0.05,0.08], "max_iter":[200,400]}):
        runs.append(("hgb", params))

    log(f"STEP 4/6: Train+Validate ({len(runs)} runs)")
    best_model = None
    best_ll = float("inf")
    best_desc = None

    pbar = tqdm(total=len(runs), desc=f"{tf} search")
    for i, (name, params) in enumerate(runs, 1):
        if name == "logreg":
            base = Pipeline([
                ("scaler", StandardScaler()),
                ("clf", LogisticRegression(max_iter=3000, class_weight="balanced", solver="lbfgs", C=float(params["C"])))
            ])
        elif name == "rf":
            base = RandomForestClassifier(
                random_state=42, n_jobs=-1, class_weight="balanced_subsample",
                n_estimators=int(params["n_estimators"]),
                max_depth=int(params["max_depth"]),
                min_samples_leaf=int(params["min_samples_leaf"])
            )
        else:
            base = HistGradientBoostingClassifier(
                random_state=42,
                max_depth=int(params["max_depth"]),
                learning_rate=float(params["learning_rate"]),
                max_iter=int(params["max_iter"])
            )

        clf = CalibratedClassifierCV(base, method="sigmoid", cv=3)
        clf.fit(Xtr, ytr)

        p_va = clf.predict_proba(Xva)[:, 1]
        ll = log_loss(yva, np.clip(p_va, 1e-6, 1-1e-6))

        if ll < best_ll:
            best_ll = ll
            best_model = clf
            best_desc = (name, params)

        pbar.set_postfix({"progress": f"{(100*i/len(runs)):.1f}%", "best_ll": f"{best_ll:.4f}", "best": best_desc[0]})
        pbar.update(1)

    pbar.close()

    log("STEP 5/6: Test metrics (BEST)")
    p_te = best_model.predict_proba(Xte)[:, 1]
    test_ll = log_loss(yte, np.clip(p_te, 1e-6, 1-1e-6))
    test_auc = roc_auc_score(yte, p_te) if len(np.unique(yte)) > 1 else float("nan")
    test_acc = accuracy_score(yte, (p_te >= 0.5).astype(int))
    cm = confusion_matrix(yte, (p_te >= 0.5).astype(int))
    rep = classification_report(yte, (p_te >= 0.5).astype(int), digits=4)
    gs = gating_stats(p_te, yte, BUY_TH, SELL_TH)

    log(f"BEST: {best_desc[0]} params={best_desc[1]}")
    log(f"VAL LogLoss={best_ll:.4f}")
    log(f"TEST LogLoss={test_ll:.4f} | AUC={test_auc:.4f} | ACC={test_acc:.4f}")
    log(f"TEST gating trades={gs['trades']} win_rate={gs['win_rate']:.2f}%")

    print("\n--- CONFUSION MATRIX (TEST) ---")
    print(cm)
    print("\n--- CLASSIFICATION REPORT (TEST) ---")
    print(rep)

    log("STEP 6/6: Save artifacts")
    tag = f"{tf}_H{H}"
    model_path = os.path.join(OUT_DIR, f"direction_model_{tag}.joblib")
    meta_path  = os.path.join(OUT_DIR, f"direction_model_{tag}.json")

    dump(best_model, model_path)
    meta = {
        "tf": tf, "horizon_bars": int(H),
        "feature_cols": FEATURE_COLS,
        "thresholds": {"buy": BUY_TH, "sell": SELL_TH},
        "best": {"name": best_desc[0], "params": best_desc[1], "val_logloss": float(best_ll)},
        "test": {"logloss": float(test_ll), "auc": float(test_auc), "acc": float(test_acc), "cm": cm.tolist(), "gating": gs},
    }
    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump(meta, f, indent=2)

    log(f"Saved model -> {model_path}")
    log(f"Saved meta  -> {meta_path}")
    return model_path, meta_path

# Run both TFs
log("=== TRAIN M15 (H=4) ===")
m15_model, m15_meta = train_tf("M15", 4)

log("=== TRAIN M30 (H=2) ===")
m30_model, m30_meta = train_tf("M30", 2)

log("DONE ✅")
(m15_model, m30_model)


[21:42:30] === TRAIN M15 (H=4) ===
[21:42:30] STEP 1/6: Load CSV for M15 (H=4)
[21:42:37] STEP 2/6: Resample -> M15
[21:42:37]    M15 candles: 124,543
[21:42:37] STEP 3/6: Features + Labels
[21:42:37]    Split: train=87,110 val=18,667 test=18,667
[21:42:37] STEP 4/6: Train+Validate (35 runs)


M15 search:   0%|          | 0/35 [00:00<?, ?it/s]

[21:50:07] STEP 5/6: Test metrics (BEST)
[21:50:08] BEST: rf params={'max_depth': 8, 'min_samples_leaf': 50, 'n_estimators': 600}
[21:50:08] VAL LogLoss=0.6923
[21:50:08] TEST LogLoss=0.6918 | AUC=0.5304 | ACC=0.5188
[21:50:08] TEST gating trades=0 win_rate=0.00%

--- CONFUSION MATRIX (TEST) ---
[[5509 3693]
 [5290 4175]]

--- CLASSIFICATION REPORT (TEST) ---
              precision    recall  f1-score   support

           0     0.5101    0.5987    0.5509      9202
           1     0.5306    0.4411    0.4817      9465

    accuracy                         0.5188     18667
   macro avg     0.5204    0.5199    0.5163     18667
weighted avg     0.5205    0.5188    0.5158     18667

[21:50:08] STEP 6/6: Save artifacts
[21:50:09] Saved model -> artifacts\direction_model_M15_H4.joblib
[21:50:09] Saved meta  -> artifacts\direction_model_M15_H4.json
[21:50:09] === TRAIN M30 (H=2) ===
[21:50:09] STEP 1/6: Load CSV for M30 (H=2)
[21:50:13] STEP 2/6: Resample -> M30
[21:50:14]    M30 candles: 62

M30 search:   0%|          | 0/35 [00:00<?, ?it/s]

[21:55:41] STEP 5/6: Test metrics (BEST)
[21:55:41] BEST: rf params={'max_depth': 8, 'min_samples_leaf': 25, 'n_estimators': 300}
[21:55:41] VAL LogLoss=0.6925
[21:55:41] TEST LogLoss=0.6923 | AUC=0.5284 | ACC=0.5199
[21:55:41] TEST gating trades=0 win_rate=0.00%

--- CONFUSION MATRIX (TEST) ---
[[3052 1548]
 [2930 1797]]

--- CLASSIFICATION REPORT (TEST) ---
              precision    recall  f1-score   support

           0     0.5102    0.6635    0.5768      4600
           1     0.5372    0.3802    0.4452      4727

    accuracy                         0.5199      9327
   macro avg     0.5237    0.5218    0.5110      9327
weighted avg     0.5239    0.5199    0.5101      9327

[21:55:41] STEP 6/6: Save artifacts
[21:55:42] Saved model -> artifacts\direction_model_M30_H2.joblib
[21:55:42] Saved meta  -> artifacts\direction_model_M30_H2.json
[21:55:42] DONE ✅


('artifacts\\direction_model_M15_H4.joblib',
 'artifacts\\direction_model_M30_H2.joblib')

In [9]:
import os, json, time, math
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss, classification_report, confusion_matrix
from joblib import dump

# =========================
# CONFIG (EDIT THESE)
# =========================
CSV_PATH = r"C:\Users\Timothy.Mandingwa\Desktop\live_app_rules\data\EURUSD_OANDA_M1.csv"
OUT_DIR = "artifacts"
os.makedirs(OUT_DIR, exist_ok=True)

# gating search constraints
MIN_TRADES_FRAC = 0.02   # at least 2% of validation bars must become trades
MIN_WIN_LIFT    = 0.00   # require >= baseline winrate + this (0.00 = no strict lift constraint)
MAX_TRADES_FRAC = 0.50   # avoid picking thresholds that basically trade everything

# gating threshold grid (you can adjust later)
BUY_GRID  = np.round(np.arange(0.52, 0.71, 0.02), 2)
SELL_GRID = np.round(np.arange(0.48, 0.29, -0.02), 2)

# fallback if spread in your CSV is weird
COST_FLOOR = 0.00000  # set to 0.00001 if you want a small minimum cost

# models search (same as your working setup)
LOGREG_C = [0.25, 0.5, 1.0, 2.0, 4.0]
RF_GRID  = {"n_estimators":[300,600], "max_depth":[6,8,10], "min_samples_leaf":[25,50,100]}
HGB_GRID = {"max_depth":[3,6], "learning_rate":[0.03,0.05,0.08], "max_iter":[200,400]}

def log(msg):
    print(f"[{time.strftime('%H:%M:%S')}] {msg}")

# -------------------------
# DATA HELPERS
# -------------------------
def ensure_dt_index(df, time_col="time"):
    if time_col in df.columns:
        df[time_col] = pd.to_datetime(df[time_col], utc=True, errors="coerce")
        df = df.dropna(subset=[time_col]).set_index(time_col)
    else:
        df.index = pd.to_datetime(df.index, utc=True, errors="coerce")
        df = df[~df.index.isna()]
    return df.sort_index()

def mid_ohlc_from_oanda(df):
    needed = ["bid_o","bid_h","bid_l","bid_c","ask_o","ask_h","ask_l","ask_c"]
    missing = [c for c in needed if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns in CSV: {missing}")

    out = pd.DataFrame(index=df.index)
    # mid prices
    out["open"]  = (pd.to_numeric(df["bid_o"], errors="coerce") + pd.to_numeric(df["ask_o"], errors="coerce")) / 2.0
    out["high"]  = (pd.to_numeric(df["bid_h"], errors="coerce") + pd.to_numeric(df["ask_h"], errors="coerce")) / 2.0
    out["low"]   = (pd.to_numeric(df["bid_l"], errors="coerce") + pd.to_numeric(df["ask_l"], errors="coerce")) / 2.0
    out["close"] = (pd.to_numeric(df["bid_c"], errors="coerce") + pd.to_numeric(df["ask_c"], errors="coerce")) / 2.0

    # spread (in price units)
    bidc = pd.to_numeric(df["bid_c"], errors="coerce")
    askc = pd.to_numeric(df["ask_c"], errors="coerce")
    out["spread"] = (askc - bidc)

    out["volume"] = pd.to_numeric(df["volume"], errors="coerce").fillna(0.0) if "volume" in df.columns else 0.0
    out = out.dropna(subset=["open","high","low","close"])
    return out

def resample_tf(m1_mid, tf):
    tf = tf.upper().strip()
    rule = {"M15":"15min", "M30":"30min"}[tf]

    o = m1_mid["open"].resample(rule).first()
    h = m1_mid["high"].resample(rule).max()
    l = m1_mid["low"].resample(rule).min()
    c = m1_mid["close"].resample(rule).last()
    v = m1_mid["volume"].resample(rule).sum()
    sp = m1_mid["spread"].resample(rule).mean()

    out = pd.DataFrame({"open":o,"high":h,"low":l,"close":c,"volume":v,"spread":sp}).dropna()
    return out

# -------------------------
# INDICATORS
# -------------------------
def ema(s, n): 
    return s.ewm(span=n, adjust=False).mean()

def rsi(close, n=14):
    d = close.diff()
    up = d.clip(lower=0).rolling(n).mean()
    dn = (-d.clip(upper=0)).rolling(n).mean()
    rs = up / (dn + 1e-12)
    return 100 - (100 / (1 + rs))

def atr(df, n=14):
    high, low, close = df["high"], df["low"], df["close"]
    prev = close.shift(1)
    tr = pd.concat([(high-low), (high-prev).abs(), (low-prev).abs()], axis=1).max(axis=1)
    return tr.rolling(n).mean()

def macd(close, fast=12, slow=26, sig=9):
    macd_line = ema(close, fast) - ema(close, slow)
    sig_line = ema(macd_line, sig)
    hist = macd_line - sig_line
    return macd_line, sig_line, hist

def bollinger(close, n=20, k=2.0):
    mid = close.rolling(n).mean()
    sd  = close.rolling(n).std()
    up  = mid + k*sd
    lo  = mid - k*sd
    width = (up - lo) / (close.abs() + 1e-12)
    z = (close - mid) / (sd + 1e-12)
    return mid, up, lo, width, z

def rolling_z(x, n=100):
    return (x - x.rolling(n).mean()) / (x.rolling(n).std() + 1e-12)

# -------------------------
# FEATURES (IMPROVED)
# -------------------------
def build_features(c: pd.DataFrame) -> pd.DataFrame:
    df = c.copy()

    # returns
    df["ret1"]  = df["close"].pct_change(1)
    df["ret4"]  = df["close"].pct_change(4)
    df["ret8"]  = df["close"].pct_change(8)
    df["ret16"] = df["close"].pct_change(16)

    # EMAs + slopes
    df["ema20"] = ema(df["close"], 20)
    df["ema50"] = ema(df["close"], 50)
    df["ema100"] = ema(df["close"], 100)

    df["ema_diff2050"] = (df["ema20"] - df["ema50"]) / (df["close"].abs() + 1e-12)
    df["ema_diff50100"] = (df["ema50"] - df["ema100"]) / (df["close"].abs() + 1e-12)

    df["ema20_slope"] = df["ema20"].diff(3) / (df["close"].abs() + 1e-12)
    df["ema50_slope"] = df["ema50"].diff(3) / (df["close"].abs() + 1e-12)

    # RSI + RSI slope
    df["rsi14"] = rsi(df["close"], 14)
    df["rsi_slope"] = df["rsi14"].diff(3)

    # MACD
    m, s, h = macd(df["close"], 12, 26, 9)
    df["macd"] = m
    df["macd_sig"] = s
    df["macd_hist"] = h
    df["macd_cross_up"] = ((df["macd"].shift(1) <= df["macd_sig"].shift(1)) & (df["macd"] > df["macd_sig"])).astype(int)
    df["macd_cross_dn"] = ((df["macd"].shift(1) >= df["macd_sig"].shift(1)) & (df["macd"] < df["macd_sig"])).astype(int)

    # ATR + range structure
    df["atr14"] = atr(df, 14)
    df["atr_pct"] = df["atr14"] / (df["close"].abs() + 1e-12)
    df["atr_z"] = rolling_z(df["atr_pct"], 200)

    df["hl_range"] = (df["high"] - df["low"]) / (df["close"].abs() + 1e-12)
    df["oc_range"] = (df["close"] - df["open"]) / (df["close"].abs() + 1e-12)
    df["body_to_range"] = (df["oc_range"].abs()) / (df["hl_range"] + 1e-12)

    # Bollinger regime + position
    bb_mid, bb_up, bb_lo, bb_width, bb_z = bollinger(df["close"], 20, 2.0)
    df["bb_width"] = bb_width
    df["bb_z"] = bb_z
    df["bb_width_z"] = rolling_z(df["bb_width"], 200)

    # Spread/Volume features
    df["spread_pct"] = df["spread"] / (df["close"].abs() + 1e-12)
    df["spread_z"] = rolling_z(df["spread_pct"], 200)

    df["vol_z"] = (df["volume"] - df["volume"].rolling(200).mean()) / (df["volume"].rolling(200).std() + 1e-12)
    df["vol_chg"] = df["volume"].pct_change(5).replace([np.inf, -np.inf], np.nan)

    # Time-of-day features (helps FX sometimes)
    # index is UTC
    hrs = df.index.hour + df.index.minute/60.0
    df["tod_sin"] = np.sin(2*np.pi*hrs/24.0)
    df["tod_cos"] = np.cos(2*np.pi*hrs/24.0)

    # Simple regime flags
    df["trend_flag"] = (df["ema20"] > df["ema50"]).astype(int)
    df["vol_flag"]   = (df["atr_z"] > 0).astype(int)

    df = df.replace([np.inf, -np.inf], np.nan)
    return df.dropna()

FEATURE_COLS = [
    "ret1","ret4","ret8","ret16",
    "ema_diff2050","ema_diff50100","ema20_slope","ema50_slope",
    "rsi14","rsi_slope",
    "macd","macd_sig","macd_hist","macd_cross_up","macd_cross_dn",
    "atr_pct","atr_z","hl_range","oc_range","body_to_range",
    "bb_width","bb_z","bb_width_z",
    "spread_pct","spread_z",
    "vol_z","vol_chg",
    "tod_sin","tod_cos",
    "trend_flag","vol_flag",
]

def make_label(df, H):
    fut = df["close"].shift(-H)
    return (fut > df["close"]).astype(int)

def future_return(df, H):
    fut = df["close"].shift(-H)
    return (fut - df["close"]) / (df["close"].abs() + 1e-12)

def time_split_idx(n, train_frac=0.70, val_frac=0.15):
    tr_end = int(n * train_frac)
    va_end = int(n * (train_frac + val_frac))
    return slice(0,tr_end), slice(tr_end,va_end), slice(va_end,n)

# -------------------------
# EXPECTANCY + GATING SEARCH
# -------------------------
def compute_expectancy(p_up, y_true, f_ret, cost, buy_th, sell_th):
    """
    Direction model gating:
      BUY if p>=buy_th
      SELL if p<=sell_th
    PnL proxy per trade:
      BUY: +future_return - cost
      SELL: -(future_return) - cost
    """
    p_up = np.asarray(p_up)
    y_true = np.asarray(y_true).astype(int)
    f_ret = np.asarray(f_ret).astype(float)
    cost = np.asarray(cost).astype(float)

    take_buy = p_up >= buy_th
    take_sell = p_up <= sell_th
    take = take_buy | take_sell

    if take.sum() == 0:
        return {
            "trades": 0,
            "win_rate": 0.0,
            "expectancy": -np.inf,
            "avg_ret": 0.0
        }

    pred_dir = np.where(take_buy, 1, 0)  # 1=UP (BUY), 0=DOWN (SELL)
    wins = (pred_dir[take] == y_true[take])
    win_rate = float(wins.mean() * 100.0)

    dir_sign = np.where(take_buy, 1.0, -1.0)
    pnl = dir_sign[take] * f_ret[take] - np.maximum(cost[take], COST_FLOOR)
    expectancy = float(np.mean(pnl))
    avg_ret = float(np.mean(dir_sign[take] * f_ret[take]))

    return {
        "trades": int(take.sum()),
        "win_rate": win_rate,
        "expectancy": expectancy,
        "avg_ret": avg_ret
    }

def baseline_stats(p_up, y_true, f_ret, cost):
    """
    Baseline = always trade with p>=0.5 BUY else SELL
    """
    p_up = np.asarray(p_up)
    y_true = np.asarray(y_true).astype(int)
    f_ret = np.asarray(f_ret).astype(float)
    cost = np.asarray(cost).astype(float)

    take_buy = p_up >= 0.5
    pred_dir = np.where(take_buy, 1, 0)
    wins = (pred_dir == y_true)
    win_rate = float(wins.mean() * 100.0)

    dir_sign = np.where(take_buy, 1.0, -1.0)
    pnl = dir_sign * f_ret - np.maximum(cost, COST_FLOOR)
    expectancy = float(np.mean(pnl))
    return {
        "trades": int(len(y_true)),
        "win_rate": win_rate,
        "expectancy": expectancy
    }

def search_best_thresholds(p_up, y_true, f_ret, cost, tf_label):
    base = baseline_stats(p_up, y_true, f_ret, cost)

    n = len(y_true)
    min_trades = max(int(n * MIN_TRADES_FRAC), 50)
    max_trades = int(n * MAX_TRADES_FRAC)

    runs = []
    for b in BUY_GRID:
        for s in SELL_GRID:
            if s >= 0.5 or b <= 0.5 or s >= b:
                continue
            runs.append((float(b), float(s)))

    best = None
    best_score = -np.inf

    pbar = tqdm(total=len(runs), desc=f"{tf_label} threshold search")
    for i, (b, s) in enumerate(runs, 1):
        st = compute_expectancy(p_up, y_true, f_ret, cost, b, s)

        # constraints
        if st["trades"] < min_trades:
            score = -np.inf
        elif st["trades"] > max_trades:
            score = -np.inf
        elif st["win_rate"] < (base["win_rate"] + MIN_WIN_LIFT):
            score = -np.inf
        else:
            # objective: maximize expectancy after costs
            score = st["expectancy"]

        if score > best_score:
            best_score = score
            best = {
                "buy": b, "sell": s,
                "stats": st,
                "baseline": base,
                "min_trades": int(min_trades),
                "max_trades": int(max_trades),
            }

        pbar.set_postfix({
            "progress": f"{(100*i/len(runs)):.1f}%",
            "best_exp": f"{(best_score if np.isfinite(best_score) else -9):.6f}",
            "best_pair": f"{best['buy']:.2f}/{best['sell']:.2f}" if best else "NA",
            "best_trades": best["stats"]["trades"] if best else 0
        })
        pbar.update(1)

    pbar.close()
    return best

# -------------------------
# TRAINING
# -------------------------
def train_tf(tf, H):
    log(f"STEP 1/7: Load CSV for {tf} (H={H})")
    raw = pd.read_csv(CSV_PATH)
    raw = ensure_dt_index(raw, "time")
    mid = mid_ohlc_from_oanda(raw)

    log(f"STEP 2/7: Resample -> {tf}")
    candles = resample_tf(mid, tf)
    log(f"   {tf} candles: {len(candles):,}")

    log("STEP 3/7: Features + Labels")
    feats = build_features(candles)
    feats["y"] = make_label(feats, H).astype(int)
    feats["f_ret"] = future_return(feats, H)
    feats = feats.dropna(subset=["y","f_ret"])

    # cost proxy: spread at bar t as pct of close
    feats["cost"] = np.maximum(feats["spread_pct"].astype(float).values, COST_FLOOR)

    X = feats[FEATURE_COLS].astype(float).values
    y = feats["y"].values.astype(int)
    f_ret = feats["f_ret"].values.astype(float)
    cost = feats["cost"].values.astype(float)

    tr, va, te = time_split_idx(len(feats))
    Xtr, ytr = X[tr], y[tr]
    Xva, yva = X[va], y[va]
    Xte, yte = X[te], y[te]

    fva, cva = f_ret[va], cost[va]
    fte, cte = f_ret[te], cost[te]

    log(f"   Split: train={len(ytr):,} val={len(yva):,} test={len(yte):,}")

    # Candidate runs
    runs = []
    for params in ParameterGrid({"C": LOGREG_C}):
        runs.append(("logreg", params))
    for params in ParameterGrid(RF_GRID):
        runs.append(("rf", params))
    for params in ParameterGrid(HGB_GRID):
        runs.append(("hgb", params))

    log(f"STEP 4/7: Train+Validate (model search: {len(runs)} runs)")
    best_model = None
    best_ll = float("inf")
    best_desc = None

    pbar = tqdm(total=len(runs), desc=f"{tf} model search")
    for i, (name, params) in enumerate(runs, 1):
        if name == "logreg":
            base = Pipeline([
                ("scaler", StandardScaler()),
                ("clf", LogisticRegression(
                    max_iter=3000, class_weight="balanced", solver="lbfgs",
                    C=float(params["C"])
                ))
            ])
        elif name == "rf":
            base = RandomForestClassifier(
                random_state=42, n_jobs=-1, class_weight="balanced_subsample",
                n_estimators=int(params["n_estimators"]),
                max_depth=int(params["max_depth"]),
                min_samples_leaf=int(params["min_samples_leaf"])
            )
        else:
            base = HistGradientBoostingClassifier(
                random_state=42,
                max_depth=int(params["max_depth"]),
                learning_rate=float(params["learning_rate"]),
                max_iter=int(params["max_iter"])
            )

        clf = CalibratedClassifierCV(base, method="sigmoid", cv=3)
        clf.fit(Xtr, ytr)

        p_va = clf.predict_proba(Xva)[:, 1]
        ll = log_loss(yva, np.clip(p_va, 1e-6, 1-1e-6))

        if ll < best_ll:
            best_ll = ll
            best_model = clf
            best_desc = (name, params)

        pbar.set_postfix({
            "progress": f"{(100*i/len(runs)):.1f}%",
            "best_ll": f"{best_ll:.4f}",
            "best": best_desc[0]
        })
        pbar.update(1)

    pbar.close()

    log("STEP 5/7: Threshold search on VALIDATION (maximize expectancy after costs)")
    p_va_best = best_model.predict_proba(Xva)[:, 1]
    best_thr = search_best_thresholds(p_va_best, yva, fva, cva, tf_label=tf)

    if best_thr is None or not np.isfinite(best_thr["stats"]["expectancy"]):
        # fallback if constraints too strict
        log("   WARNING: No threshold pair met constraints. Falling back to buy=0.55 sell=0.45")
        best_thr = {
            "buy": 0.55, "sell": 0.45,
            "stats": compute_expectancy(p_va_best, yva, fva, cva, 0.55, 0.45),
            "baseline": baseline_stats(p_va_best, yva, fva, cva),
            "min_trades": int(max(int(len(yva)*MIN_TRADES_FRAC), 50)),
            "max_trades": int(len(yva)*MAX_TRADES_FRAC)
        }

    log(f"   VAL baseline: win_rate={best_thr['baseline']['win_rate']:.2f}% exp={best_thr['baseline']['expectancy']:.6f}")
    log(f"   VAL best gate: buy={best_thr['buy']:.2f} sell={best_thr['sell']:.2f} "
        f"trades={best_thr['stats']['trades']} win_rate={best_thr['stats']['win_rate']:.2f}% "
        f"exp={best_thr['stats']['expectancy']:.6f}")

    log("STEP 6/7: Test metrics + gated performance (using chosen thresholds)")
    p_te = best_model.predict_proba(Xte)[:, 1]

    test_ll  = log_loss(yte, np.clip(p_te, 1e-6, 1-1e-6))
    test_auc = roc_auc_score(yte, p_te) if len(np.unique(yte)) > 1 else float("nan")
    test_acc = accuracy_score(yte, (p_te >= 0.5).astype(int))
    cm = confusion_matrix(yte, (p_te >= 0.5).astype(int))
    rep = classification_report(yte, (p_te >= 0.5).astype(int), digits=4)

    base_te = baseline_stats(p_te, yte, fte, cte)
    gate_te = compute_expectancy(p_te, yte, fte, cte, best_thr["buy"], best_thr["sell"])

    log(f"BEST MODEL: {best_desc[0]} params={best_desc[1]}")
    log(f"VAL LogLoss={best_ll:.4f}")
    log(f"TEST LogLoss={test_ll:.4f} | AUC={test_auc:.4f} | ACC={test_acc:.4f}")

    log(f"TEST baseline: trades={base_te['trades']} win_rate={base_te['win_rate']:.2f}% exp={base_te['expectancy']:.6f}")
    log(f"TEST gated   : buy={best_thr['buy']:.2f} sell={best_thr['sell']:.2f} "
        f"trades={gate_te['trades']} win_rate={gate_te['win_rate']:.2f}% exp={gate_te['expectancy']:.6f}")

    print("\n--- CONFUSION MATRIX (TEST @ 0.5 cutoff) ---")
    print(cm)
    print("\n--- CLASSIFICATION REPORT (TEST @ 0.5 cutoff) ---")
    print(rep)

    log("STEP 7/7: Save artifacts")
    tag = f"{tf}_H{H}"
    model_path = os.path.join(OUT_DIR, f"direction_model_{tag}.joblib")
    meta_path  = os.path.join(OUT_DIR, f"direction_model_{tag}.json")

    dump(best_model, model_path)

    meta = {
        "tf": tf,
        "horizon_bars": int(H),
        "feature_cols": FEATURE_COLS,

        "best_model": {
            "name": best_desc[0],
            "params": best_desc[1],
            "val_logloss": float(best_ll),
        },

        "threshold_search": {
            "buy_grid": BUY_GRID.tolist(),
            "sell_grid": SELL_GRID.tolist(),
            "min_trades_frac": float(MIN_TRADES_FRAC),
            "max_trades_frac": float(MAX_TRADES_FRAC),
            "min_win_lift": float(MIN_WIN_LIFT),
        },

        "thresholds_best": {
            "buy": float(best_thr["buy"]),
            "sell": float(best_thr["sell"]),
            "val": best_thr["stats"],
            "val_baseline": best_thr["baseline"],
            "constraints": {"min_trades": int(best_thr["min_trades"]), "max_trades": int(best_thr["max_trades"])},
        },

        "test": {
            "logloss": float(test_ll),
            "auc": float(test_auc),
            "acc": float(test_acc),
            "cm": cm.tolist(),
            "baseline": base_te,
            "gated": {
                "buy": float(best_thr["buy"]),
                "sell": float(best_thr["sell"]),
                "stats": gate_te
            }
        }
    }

    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump(meta, f, indent=2)

    log(f"Saved model -> {model_path}")
    log(f"Saved meta  -> {meta_path}")

    return model_path, meta_path

# -------------------------
# RUN BOTH TFs
# -------------------------
log("=== TRAIN M15 (H=4) ===")
m15_model, m15_meta = train_tf("M15", 4)

log("=== TRAIN M30 (H=2) ===")
m30_model, m30_meta = train_tf("M30", 2)

log("DONE ✅")
(m15_model, m30_model)


[22:31:17] === TRAIN M15 (H=4) ===
[22:31:17] STEP 1/7: Load CSV for M15 (H=4)
[22:31:22] STEP 2/7: Resample -> M15
[22:31:23]    M15 candles: 124,543
[22:31:23] STEP 3/7: Features + Labels
[22:31:23]    Split: train=87,024 val=18,648 test=18,649
[22:31:23] STEP 4/7: Train+Validate (model search: 35 runs)


M15 model search:   0%|          | 0/35 [00:00<?, ?it/s]

[22:41:57] STEP 5/7: Threshold search on VALIDATION (maximize expectancy after costs)


M15 threshold search:   0%|          | 0/100 [00:00<?, ?it/s]

[22:41:58]    VAL baseline: win_rate=52.36% exp=-0.000130
[22:41:58]    VAL best gate: buy=0.54 sell=0.46 trades=571 win_rate=63.40% exp=-0.000098
[22:41:58] STEP 6/7: Test metrics + gated performance (using chosen thresholds)
[22:41:58] BEST MODEL: rf params={'max_depth': 8, 'min_samples_leaf': 25, 'n_estimators': 600}
[22:41:58] VAL LogLoss=0.6917
[22:41:58] TEST LogLoss=0.6913 | AUC=0.5348 | ACC=0.5219
[22:41:59] TEST baseline: trades=18649 win_rate=52.19% exp=-0.000132
[22:41:59] TEST gated   : buy=0.54 sell=0.46 trades=536 win_rate=65.86% exp=-0.000166

--- CONFUSION MATRIX (TEST @ 0.5 cutoff) ---
[[6131 3066]
 [5851 3601]]

--- CLASSIFICATION REPORT (TEST @ 0.5 cutoff) ---
              precision    recall  f1-score   support

           0     0.5117    0.6666    0.5790      9197
           1     0.5401    0.3810    0.4468      9452

    accuracy                         0.5219     18649
   macro avg     0.5259    0.5238    0.5129     18649
weighted avg     0.5261    0.5219    0.5

M30 model search:   0%|          | 0/35 [00:00<?, ?it/s]

[22:47:11] STEP 5/7: Threshold search on VALIDATION (maximize expectancy after costs)


M30 threshold search:   0%|          | 0/100 [00:00<?, ?it/s]

[22:47:12]    VAL baseline: win_rate=51.47% exp=-0.000145
[22:47:12]    VAL best gate: buy=0.52 sell=0.44 trades=340 win_rate=61.18% exp=-0.000073
[22:47:12] STEP 6/7: Test metrics + gated performance (using chosen thresholds)
[22:47:12] BEST MODEL: rf params={'max_depth': 10, 'min_samples_leaf': 100, 'n_estimators': 300}
[22:47:12] VAL LogLoss=0.6922
[22:47:12] TEST LogLoss=0.6921 | AUC=0.5290 | ACC=0.5213
[22:47:12] TEST baseline: trades=9309 win_rate=52.13% exp=-0.000135
[22:47:12] TEST gated   : buy=0.52 sell=0.44 trades=311 win_rate=65.27% exp=-0.000171

--- CONFUSION MATRIX (TEST @ 0.5 cutoff) ---
[[3194 1400]
 [3056 1659]]

--- CLASSIFICATION REPORT (TEST @ 0.5 cutoff) ---
              precision    recall  f1-score   support

           0     0.5110    0.6953    0.5891      4594
           1     0.5423    0.3519    0.4268      4715

    accuracy                         0.5213      9309
   macro avg     0.5267    0.5236    0.5079      9309
weighted avg     0.5269    0.5213    0.

('artifacts\\direction_model_M15_H4.joblib',
 'artifacts\\direction_model_M30_H2.joblib')

In [10]:
import os, json, time, math
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import ParameterGrid, TimeSeriesSplit
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss, classification_report, confusion_matrix
from joblib import dump

# =========================
# CONFIG
# =========================
CSV_PATH = r"C:\Users\Timothy.Mandingwa\Desktop\live_app_rules\data\EURUSD_OANDA_M1.csv"
OUT_DIR = "artifacts_v2"
os.makedirs(OUT_DIR, exist_ok=True)

# realistic-ish costs (in price terms) - tune these to match your broker
# We'll model cost per trade as: cost = spread_mean * cost_mult
COST_MULT = 1.0   # 1.0 means "pay the spread once" (conservative enough for gating tests)

# threshold search
BUY_GRID  = np.linspace(0.50, 0.75, 26)  # 0.50..0.75 step 0.01
SELL_GRID = np.linspace(0.25, 0.50, 26)  # 0.25..0.50 step 0.01
MIN_TRADES_VAL = 250     # require enough validation trades
MIN_TRADES_TEST = 250    # require enough test trades

# time split
TRAIN_FRAC = 0.70
VAL_FRAC   = 0.15

SEED = 42

def log(msg):
    print(f"[{time.strftime('%H:%M:%S')}] {msg}")

def ensure_dt_index(df, time_col="time"):
    if time_col in df.columns:
        df[time_col] = pd.to_datetime(df[time_col], utc=True, errors="coerce")
        df = df.dropna(subset=[time_col]).set_index(time_col)
    else:
        df.index = pd.to_datetime(df.index, utc=True, errors="coerce")
        df = df[~df.index.isna()]
    return df.sort_index()

def mid_ohlc_from_oanda(df):
    needed = ["bid_o","bid_h","bid_l","bid_c","ask_o","ask_h","ask_l","ask_c"]
    missing = [c for c in needed if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns in CSV: {missing}")

    out = pd.DataFrame(index=df.index)
    # robust numeric conversion
    for c in needed + (["volume"] if "volume" in df.columns else []):
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    out["open"]  = (df["bid_o"] + df["ask_o"]) / 2.0
    out["high"]  = (df["bid_h"] + df["ask_h"]) / 2.0
    out["low"]   = (df["bid_l"] + df["ask_l"]) / 2.0
    out["close"] = (df["bid_c"] + df["ask_c"]) / 2.0
    out["spread"] = (df["ask_c"] - df["bid_c"]).fillna(0.0)
    out["volume"] = df["volume"].fillna(0.0) if "volume" in df.columns else 0.0
    return out.dropna()

def resample_tf(m1_mid, tf):
    tf = tf.upper().strip()
    rule = {"M15":"15min", "M30":"30min"}[tf]
    o = m1_mid["open"].resample(rule).first()
    h = m1_mid["high"].resample(rule).max()
    l = m1_mid["low"].resample(rule).min()
    c = m1_mid["close"].resample(rule).last()
    v = m1_mid["volume"].resample(rule).sum()
    sp = m1_mid["spread"].resample(rule).mean()
    return pd.DataFrame({"open":o,"high":h,"low":l,"close":c,"volume":v,"spread":sp}).dropna()

def ema(s, n): 
    return s.ewm(span=n, adjust=False).mean()

def rsi(close, n=14):
    d = close.diff()
    up = d.clip(lower=0).rolling(n).mean()
    dn = (-d.clip(upper=0)).rolling(n).mean()
    rs = up / (dn + 1e-12)
    return 100 - (100 / (1 + rs))

def atr(df, n=14):
    high, low, close = df["high"], df["low"], df["close"]
    prev = close.shift(1)
    tr = pd.concat([(high-low), (high-prev).abs(), (low-prev).abs()], axis=1).max(axis=1)
    return tr.rolling(n).mean()

def macd(close, fast=12, slow=26, sig=9):
    macd_line = ema(close, fast) - ema(close, slow)
    sig_line = ema(macd_line, sig)
    hist = macd_line - sig_line
    return macd_line, sig_line, hist

def rolling_z(x, n):
    mu = x.rolling(n).mean()
    sd = x.rolling(n).std()
    return (x - mu) / (sd + 1e-12)

def build_features(c):
    df = c.copy()

    # returns (multi-lag)
    for k in [1,2,3,4,6,8,12]:
        df[f"ret{k}"] = df["close"].pct_change(k)

    # trend & slope
    df["ema20"] = ema(df["close"], 20)
    df["ema50"] = ema(df["close"], 50)
    df["ema100"] = ema(df["close"], 100)
    df["ema_diff_20_50"] = (df["ema20"] - df["ema50"]) / (df["close"].abs() + 1e-12)
    df["ema_diff_50_100"] = (df["ema50"] - df["ema100"]) / (df["close"].abs() + 1e-12)
    df["ema20_slope"] = df["ema20"].diff() / (df["close"].abs() + 1e-12)

    # mean-reversion position
    df["z_close_50"] = rolling_z(df["close"], 50)
    df["z_ret_50"] = rolling_z(df["ret1"], 50)

    # momentum
    df["rsi14"] = rsi(df["close"], 14)
    df["rsi_slope"] = df["rsi14"].diff()

    m, s, h = macd(df["close"], 12, 26, 9)
    df["macd"] = m
    df["macd_sig"] = s
    df["macd_hist"] = h
    df["macd_hist_slope"] = df["macd_hist"].diff()

    # volatility regime
    df["atr14"] = atr(df, 14)
    df["atr_pct"] = df["atr14"] / (df["close"].abs() + 1e-12)
    df["atr_pct_z"] = rolling_z(df["atr_pct"], 200)

    df["hl_range"] = (df["high"] - df["low"]) / (df["close"].abs() + 1e-12)
    df["oc_range"] = (df["close"] - df["open"]) / (df["close"].abs() + 1e-12)
    df["range_z"] = rolling_z(df["hl_range"], 200)

    # spread & volume
    df["spread_pct"] = df["spread"] / (df["close"].abs() + 1e-12)
    df["spread_z"] = rolling_z(df["spread_pct"], 200)
    df["vol_z"] = rolling_z(df["volume"], 200)

    # time-of-day features (helps FX sometimes)
    idx = df.index
    df["hour"] = idx.hour
    df["dow"] = idx.dayofweek
    df["sin_hour"] = np.sin(2*np.pi*df["hour"]/24.0)
    df["cos_hour"] = np.cos(2*np.pi*df["hour"]/24.0)
    df["sin_dow"] = np.sin(2*np.pi*df["dow"]/7.0)
    df["cos_dow"] = np.cos(2*np.pi*df["dow"]/7.0)

    return df.dropna()

def make_label(df, H):
    fut = df["close"].shift(-H)
    return (fut > df["close"]).astype(int)

def time_split_idx(n, train_frac=TRAIN_FRAC, val_frac=VAL_FRAC):
    tr_end = int(n * train_frac)
    va_end = int(n * (train_frac + val_frac))
    return slice(0,tr_end), slice(tr_end,va_end), slice(va_end,n)

def expectancy_after_costs(df_eval, p_up, y_true, H, buy_th, sell_th, cost_mult=COST_MULT):
    """
    Uses future return over H bars as "trade result":
      BUY:  ret = close[t+H]/close[t] - 1
      SELL: ret = -(close[t+H]/close[t] - 1)
    Costs: subtract spread_mean * cost_mult (in return terms).
    """
    p_up = np.asarray(p_up)
    y_true = np.asarray(y_true).astype(int)

    take_buy = p_up >= buy_th
    take_sell = p_up <= sell_th
    take = take_buy | take_sell
    ntr = int(take.sum())
    if ntr == 0:
        return None

    close = df_eval["close"].values
    fut = np.roll(close, -H)
    raw_ret = (fut / (close + 1e-12)) - 1.0
    raw_ret[-H:] = np.nan  # last H have no future

    dirn = np.where(take_buy, 1, -1)  # +1 buy, -1 sell
    trade_ret = raw_ret * dirn
    trade_ret = trade_ret[take]
    trade_ret = trade_ret[np.isfinite(trade_ret)]
    if trade_ret.size == 0:
        return None

    # cost in return terms
    spread = df_eval["spread"].values
    spread_ret = (spread / (close + 1e-12)) * float(cost_mult)
    spread_ret = spread_ret[take]
    spread_ret = spread_ret[np.isfinite(spread_ret)]
    if spread_ret.size != trade_ret.size:
        m = min(spread_ret.size, trade_ret.size)
        trade_ret = trade_ret[:m]
        spread_ret = spread_ret[:m]

    net = trade_ret - spread_ret

    wins = (net > 0)
    win_rate = float(wins.mean() * 100.0)
    avg_win = float(net[wins].mean()) if wins.any() else 0.0
    avg_loss = float(net[~wins].mean()) if (~wins).any() else 0.0
    exp = float(net.mean())
    return {
        "trades": int(net.size),
        "win_rate": win_rate,
        "avg_win": avg_win,
        "avg_loss": avg_loss,
        "expectancy": exp,
    }

def threshold_search(df_val, p_val, y_val, H):
    best = None
    pairs = [(b,s) for b in BUY_GRID for s in SELL_GRID if b > s]
    pbar = tqdm(total=len(pairs), desc="Threshold search (VAL)")
    for i, (b, s) in enumerate(pairs, 1):
        st = expectancy_after_costs(df_val, p_val, y_val, H, b, s)
        if st is None:
            pbar.update(1); continue
        if st["trades"] < MIN_TRADES_VAL:
            pbar.update(1); continue

        # objective: maximize expectancy; tie-breaker: higher trades; then higher win_rate
        key = (st["expectancy"], st["trades"], st["win_rate"])
        if (best is None) or (key > best["key"]):
            best = {"buy": float(b), "sell": float(s), "stats": st, "key": key}

        if best is not None:
            pbar.set_postfix({
                "best_exp": f"{best['stats']['expectancy']:.6f}",
                "best_pair": f"{best['buy']:.2f}/{best['sell']:.2f}",
                "trades": best["stats"]["trades"],
                "wr%": f"{best['stats']['win_rate']:.1f}",
            })
        pbar.update(1)
    pbar.close()
    return best

def train_tf(tf, H):
    log(f"=== TRAIN {tf} (H={H}) ===")
    log("STEP 1/7: Load CSV")
    raw = pd.read_csv(CSV_PATH)
    raw = ensure_dt_index(raw, "time")
    mid = mid_ohlc_from_oanda(raw)

    log(f"STEP 2/7: Resample -> {tf}")
    candles = resample_tf(mid, tf)
    log(f"   {tf} candles: {len(candles):,}")

    log("STEP 3/7: Features + Labels")
    feats = build_features(candles)
    feats["y"] = make_label(feats, H)
    feats = feats.dropna(subset=["y"])
    feats["y"] = feats["y"].astype(int)

    feature_cols = [c for c in feats.columns if c not in ("y",)]
    X = feats[feature_cols].astype(float).values
    y = feats["y"].values

    tr, va, te = time_split_idx(len(feats))
    Xtr, ytr = X[tr], y[tr]
    Xva, yva = X[va], y[va]
    Xte, yte = X[te], y[te]

    df_tr = feats.iloc[tr]
    df_va = feats.iloc[va]
    df_te = feats.iloc[te]

    log(f"   Split: train={len(ytr):,} val={len(yva):,} test={len(yte):,}")

    # candidate models (kept tight so it runs)
    runs = []
    runs += [("logreg", p) for p in ParameterGrid({"C":[0.25,0.5,1.0,2.0,4.0]})]
    runs += [("rf", p) for p in ParameterGrid({"n_estimators":[300,600], "max_depth":[6,8,10], "min_samples_leaf":[25,50,100]})]
    runs += [("et", p) for p in ParameterGrid({"n_estimators":[600], "max_depth":[8,10], "min_samples_leaf":[25,50]})]
    runs += [("hgb", p) for p in ParameterGrid({"max_depth":[3,6], "learning_rate":[0.03,0.05,0.08], "max_iter":[200,400]})]

    log(f"STEP 4/7: Train+Validate (model search: {len(runs)} runs)")
    best_model = None
    best_ll = float("inf")
    best_desc = None

    # IMPORTANT: time-aware calibration (reduces leakage)
    tscv = TimeSeriesSplit(n_splits=3)

    pbar = tqdm(total=len(runs), desc=f"{tf} model search")
    for i, (name, params) in enumerate(runs, 1):
        if name == "logreg":
            base = Pipeline([
                ("scaler", StandardScaler()),
                ("clf", LogisticRegression(
                    max_iter=4000, class_weight="balanced",
                    solver="lbfgs", C=float(params["C"])
                ))
            ])
        elif name == "rf":
            base = RandomForestClassifier(
                random_state=SEED, n_jobs=-1, class_weight="balanced_subsample",
                n_estimators=int(params["n_estimators"]),
                max_depth=int(params["max_depth"]),
                min_samples_leaf=int(params["min_samples_leaf"])
            )
        elif name == "et":
            base = ExtraTreesClassifier(
                random_state=SEED, n_jobs=-1, class_weight="balanced",
                n_estimators=int(params["n_estimators"]),
                max_depth=int(params["max_depth"]),
                min_samples_leaf=int(params["min_samples_leaf"])
            )
        else:
            base = HistGradientBoostingClassifier(
                random_state=SEED,
                max_depth=int(params["max_depth"]),
                learning_rate=float(params["learning_rate"]),
                max_iter=int(params["max_iter"])
            )

        clf = CalibratedClassifierCV(base, method="sigmoid", cv=tscv)
        clf.fit(Xtr, ytr)

        p_va = clf.predict_proba(Xva)[:, 1]
        ll = log_loss(yva, np.clip(p_va, 1e-6, 1-1e-6))

        if ll < best_ll:
            best_ll = ll
            best_model = clf
            best_desc = (name, params)

        pbar.set_postfix({"best_ll": f"{best_ll:.4f}", "best": best_desc[0]})
        pbar.update(1)
    pbar.close()

    log("STEP 5/7: Threshold search on VALIDATION (maximize expectancy after costs)")
    p_va = best_model.predict_proba(Xva)[:, 1]
    best_gate = threshold_search(df_va, p_va, yva, H)

    # baseline = trade everything with direction = (p>=0.5) and include costs
    baseline = expectancy_after_costs(df_va, p_va, yva, H, buy_th=0.5, sell_th=0.5)  # (everything)
    if baseline is None:
        baseline = {"trades":0, "win_rate":0.0, "avg_win":0.0, "avg_loss":0.0, "expectancy":0.0}

    log(f"VAL baseline: trades={baseline['trades']} win_rate={baseline['win_rate']:.2f}% exp={baseline['expectancy']:.6f}")
    if best_gate:
        log(f"VAL best gate: buy={best_gate['buy']:.2f} sell={best_gate['sell']:.2f} "
            f"trades={best_gate['stats']['trades']} win_rate={best_gate['stats']['win_rate']:.2f}% "
            f"exp={best_gate['stats']['expectancy']:.6f}")
    else:
        log("VAL best gate: NONE (could not satisfy min-trades constraint)")

    log("STEP 6/7: Test metrics + gated performance")
    p_te = best_model.predict_proba(Xte)[:, 1]
    test_ll = log_loss(yte, np.clip(p_te, 1e-6, 1-1e-6))
    test_auc = roc_auc_score(yte, p_te) if len(np.unique(yte)) > 1 else float("nan")
    test_acc = accuracy_score(yte, (p_te >= 0.5).astype(int))

    cm = confusion_matrix(yte, (p_te >= 0.5).astype(int))
    rep = classification_report(yte, (p_te >= 0.5).astype(int), digits=4)

    test_baseline = expectancy_after_costs(df_te, p_te, yte, H, buy_th=0.5, sell_th=0.5) or {"trades":0,"win_rate":0,"avg_win":0,"avg_loss":0,"expectancy":0}
    test_gated = None
    if best_gate:
        test_gated = expectancy_after_costs(df_te, p_te, yte, H, best_gate["buy"], best_gate["sell"])
        if test_gated and test_gated["trades"] < MIN_TRADES_TEST:
            test_gated = None

    log(f"BEST MODEL: {best_desc[0]} params={best_desc[1]}")
    log(f"VAL LogLoss={best_ll:.4f}")
    log(f"TEST LogLoss={test_ll:.4f} | AUC={test_auc:.4f} | ACC={test_acc:.4f}")
    log(f"TEST baseline: trades={test_baseline['trades']} win_rate={test_baseline['win_rate']:.2f}% exp={test_baseline['expectancy']:.6f}")
    if test_gated:
        log(f"TEST gated   : buy={best_gate['buy']:.2f} sell={best_gate['sell']:.2f} "
            f"trades={test_gated['trades']} win_rate={test_gated['win_rate']:.2f}% exp={test_gated['expectancy']:.6f}")
    else:
        log("TEST gated   : NONE (insufficient trades or no valid gate)")

    print("\n--- CONFUSION MATRIX (TEST @ 0.5) ---")
    print(cm)
    print("\n--- CLASSIFICATION REPORT (TEST @ 0.5) ---")
    print(rep)

    log("STEP 7/7: Save artifacts")
    tag = f"{tf}_H{H}"
    model_path = os.path.join(OUT_DIR, f"direction_model_{tag}.joblib")
    meta_path  = os.path.join(OUT_DIR, f"direction_model_{tag}.json")

    dump(best_model, model_path)

    meta = {
        "tf": tf,
        "horizon_bars": int(H),
        "feature_cols": feature_cols,
        "cost_mult": float(COST_MULT),
        "best_model": {"name": best_desc[0], "params": best_desc[1], "val_logloss": float(best_ll)},
        "thresholds": ({"buy": best_gate["buy"], "sell": best_gate["sell"]} if best_gate else None),
        "val": {"baseline": baseline, "best_gate": (best_gate["stats"] if best_gate else None)},
        "test": {
            "logloss": float(test_ll), "auc": float(test_auc), "acc": float(test_acc),
            "cm": cm.tolist(),
            "baseline": test_baseline,
            "gated": test_gated,
        }
    }
    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump(meta, f, indent=2)

    log(f"Saved model -> {model_path}")
    log(f"Saved meta  -> {meta_path}")
    return model_path, meta_path

# Run both TFs (Option B)
m15_model, m15_meta = train_tf("M15", 4)
m30_model, m30_meta = train_tf("M30", 2)

log("DONE ✅")
(m15_model, m30_model)


[23:11:13] === TRAIN M15 (H=4) ===
[23:11:13] STEP 1/7: Load CSV
[23:11:18] STEP 2/7: Resample -> M15
[23:11:18]    M15 candles: 124,543
[23:11:18] STEP 3/7: Features + Labels
[23:11:19]    Split: train=87,031 val=18,650 test=18,650
[23:11:19] STEP 4/7: Train+Validate (model search: 39 runs)


M15 model search:   0%|          | 0/39 [00:00<?, ?it/s]

[23:21:07] STEP 5/7: Threshold search on VALIDATION (maximize expectancy after costs)


Threshold search (VAL):   0%|          | 0/675 [00:00<?, ?it/s]

[23:21:08] VAL baseline: trades=18646 win_rate=41.06% exp=-0.000133
[23:21:08] VAL best gate: buy=0.54 sell=0.43 trades=1099 win_rate=36.40% exp=-0.000097
[23:21:08] STEP 6/7: Test metrics + gated performance
[23:21:08] BEST MODEL: rf params={'max_depth': 6, 'min_samples_leaf': 100, 'n_estimators': 600}
[23:21:08] VAL LogLoss=0.6915
[23:21:08] TEST LogLoss=0.6915 | AUC=0.5286 | ACC=0.5182
[23:21:08] TEST baseline: trades=18646 win_rate=41.09% exp=-0.000136
[23:21:08] TEST gated   : buy=0.54 sell=0.43 trades=1403 win_rate=41.05% exp=-0.000111

--- CONFUSION MATRIX (TEST @ 0.5) ---
[[5046 4155]
 [4830 4619]]

--- CLASSIFICATION REPORT (TEST @ 0.5) ---
              precision    recall  f1-score   support

           0     0.5109    0.5484    0.5290      9201
           1     0.5264    0.4888    0.5069      9449

    accuracy                         0.5182     18650
   macro avg     0.5187    0.5186    0.5180     18650
weighted avg     0.5188    0.5182    0.5178     18650

[23:21:08] STEP

M30 model search:   0%|          | 0/39 [00:00<?, ?it/s]

[23:27:32] STEP 5/7: Threshold search on VALIDATION (maximize expectancy after costs)


Threshold search (VAL):   0%|          | 0/675 [00:00<?, ?it/s]

[23:27:33] VAL baseline: trades=9308 win_rate=40.86% exp=-0.000134
[23:27:33] VAL best gate: buy=0.55 sell=0.47 trades=877 win_rate=41.62% exp=-0.000056
[23:27:33] STEP 6/7: Test metrics + gated performance
[23:27:34] BEST MODEL: rf params={'max_depth': 10, 'min_samples_leaf': 100, 'n_estimators': 300}
[23:27:34] VAL LogLoss=0.6916
[23:27:34] TEST LogLoss=0.6922 | AUC=0.5253 | ACC=0.5194
[23:27:34] TEST baseline: trades=9308 win_rate=41.46% exp=-0.000130
[23:27:34] TEST gated   : buy=0.55 sell=0.47 trades=1823 win_rate=40.21% exp=-0.000147

--- CONFUSION MATRIX (TEST @ 0.5) ---
[[2836 1760]
 [2714 2000]]

--- CLASSIFICATION REPORT (TEST @ 0.5) ---
              precision    recall  f1-score   support

           0     0.5110    0.6171    0.5590      4596
           1     0.5319    0.4243    0.4720      4714

    accuracy                         0.5194      9310
   macro avg     0.5215    0.5207    0.5155      9310
weighted avg     0.5216    0.5194    0.5150      9310

[23:27:34] STEP 7

('artifacts_v2\\direction_model_M15_H4.joblib',
 'artifacts_v2\\direction_model_M30_H2.joblib')